In [32]:
import os
#import modal
#import great_expectations as ge
#import hopsworks
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#project = hopsworks.login()
#fs = project.get_feature_store()

titanic_df = pd.read_csv("https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/titanic.csv")
#X_train, X_test, y_train, y_test = train_test_split(titanic_df, test_size=0.2)

# Print shapes of train and test data
#print(X_train.shape, X_test.shape)
#print(y_train.shape, y_test.shape)
# Exploratory analysis

# First instances
print("-"*30)
print("Glimpse of the DF:")
display(titanic_df.head(5))

# Descriptive stats for titanic df
print("-"*30)
print("Statistical summary of titanic DF:")
display(titanic_df.describe())

# Find NaNs
# Age - 177 and Cabin - 687 and embarked - 2
# ML algos doesn't like missing values so we need to handle them
print("-"*30)
print("Finding NaN values:")
display(titanic_df.isnull().sum())

# I didn't do any proper feature engineering, instead I found an article mentioning 
# that these are the most important features for this dataset:
# Sex, Age, Fare, Pclass
# I drop the less important features and Cabin (since it has so many missing values)

# Drop less important columns
titanic_df.drop(['PassengerId', 'Name', 'Ticket', 'SibSp', 'Parch', 'Cabin', 'Embarked'], axis=1, inplace=True)
print(titanic_df.head(5))
display(titanic_df.columns) # THIS is what we will put in our feature store! Not sure if passanger ID needs to be included.

# Handle missing values (age)
titanic_df['Age'].fillna((titanic_df['Age'].mean()), inplace=True)
display(titanic_df.isnull().sum()) # it worked

# Map to numeric values (gender)
categories = {"female": 1, "male": 0}
titanic_df['Sex']= titanic_df['Sex'].map(categories)

# Check that all columns are numerical
display(titanic_df.dtypes)

------------------------------
Glimpse of the DF:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


------------------------------
Statistical summary of titanic DF:


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


------------------------------
Finding NaN values:


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

   Survived  Pclass     Sex   Age     Fare
0         0       3    male  22.0   7.2500
1         1       1  female  38.0  71.2833
2         1       3  female  26.0   7.9250
3         1       1  female  35.0  53.1000
4         0       3    male  35.0   8.0500


Index(['Survived', 'Pclass', 'Sex', 'Age', 'Fare'], dtype='object')

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
dtype: int64

Survived      int64
Pclass        int64
Sex           int64
Age         float64
Fare        float64
dtype: object

In [33]:
# Split the data into train and test. Then we perform normalization.

# Keeping and dropping label. We save it later for seeing how well the model predicted
labels = titanic_df['Survived']
titanic_df = titanic_df.drop('Survived', axis=1)  # Dropping label to normalize

# Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(titanic_df, labels, test_size=0.2, random_state=32)

print(X_train.shape)# Training data
print(X_test.shape) # Test set
print(y_train.shape) # labels for training data
print(y_test.shape) # labels for test set


# Normalize the data so that the model converges faster 
# convergence = a state when the loss is around the final value

# Normalizing using MinMaxScaler
# transforms numeric values to a given range. Default = 0,1 which we use
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(X_train)
scaled_test = scaler.transform(X_test)

scaled_train = pd.DataFrame(scaled_train, columns=X_train.columns, index=X_train.index)
display(scaled_train.head(5))

# TODO: Train model, split data, make predictions, evaluate, hyperparam tuning (approx 1 h of work), write comments!

(712, 4)
(179, 4)
(712,)
(179,)


,Pclass,Sex,Age,Fare
293,1.0,1.0,0.296306,0.017274
617,1.0,1.0,0.321438,0.031425
424,1.0,0.0,0.220910,0.039452
888,1.0,1.0,0.367921,0.045771
230,0.0,1.0,0.434531,0.162932


In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("   Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))
print(classification_report(y_test,y_pred))

display(y_test)

print(y_pred)

   Accuracy: 0.8156424581005587
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       108
           1       0.79      0.73      0.76        71

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.82      0.81       179



837    0
334    1
849    1
778    0
413    0
      ..
74     1
700    1
770    0
559    1
782    0
Name: Survived, Length: 179, dtype: int64

[0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1
 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 0 0
 0 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1
 1 1 1 0 0 0 1 1 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1]
